Authors  
 -  Alonso Guerrero Castaneda (UID: 1194613)  
 -  Eli Gnesin (UID: 1172961)  
 -  Tommy Misikoff (UID: 1166813)  
 -  Sanskriti Purohit (UID: 1179957)  
 -  Will Tirone (UID: 1130904)  

TA: Rick Presman 

### Question 1 {-}

Consider data ${(T_i, \delta_i)}_{i=1}^n$ as data and consider the Kaplan-Meier estimator $\hat{S}_n(t) = \prod_{j:t_j < t}(1 - \frac{d_j}{n_j})$ where $n_j$ are the number of subjects still alive at time $t_j$ and $d_j$ are the number of subjects who died at time $t_j$ and $j$ is the index of observed event times. 

### Question 2 {-}

Consider observed iid data ${(\mathbf{X}_i, Y_i)}_{i=1}^n$ and with $\lambda > 0$, define $(\hat{\mathbf{u}}_n, \hat{\mathbf{v}}_n) = \text{arg min}_{\mathbf{(u,v)}} \mathbb{P}_n \{Y - (\mathbf{u}\circ\mathbf{v})^T\mathbf{X}\}^2 + \lambda||\mathbf{u}||/2 + \lambda||\mathbf{v}||/2$.  

Then:
\begin{aligned}

\end{aligned}

### Resources and Notes {-}